In [1]:
var csv = require('csv-parser');  

In [2]:
var fs = require('fs');

In [3]:
var csv_train_file_path = './data/train.csv'

In [4]:
var X = []
var toxic_y = []
var severe_toxic_y = []

fs.createReadStream(csv_train_file_path)  
  .pipe(csv())
  .on('data', (row) => {
    X.push(row.comment_text);
    toxic_y.push(row.toxic)
    severe_toxic_y.push(row.severe_toxic)
  })
  .on('end', () => {
    console.log('CSV file successfully processed');
  });

CsvParser {
  _readableState:
   ReadableState {
     objectMode: true,
     highWaterMark: 16,
     buffer: BufferList { head: null, tail: null, length: 0 },
     length: 0,
     pipes: null,
     pipesCount: 0,
     flowing: true,
     ended: false,
     endEmitted: false,
     reading: false,
     sync: false,
     needReadable: false,
     emittedReadable: false,
     readableListening: false,
     resumeScheduled: true,
     paused: false,
     emitClose: true,
     autoDestroy: false,
     destroyed: false,
     defaultEncoding: 'utf8',
     awaitDrain: 0,
     readingMore: false,
     decoder: null,
     encoding: null },
  readable: true,
  _events:
   [Object: null prototype] {
     prefinish: [Function: prefinish],
     unpipe: [Function: onunpipe],
     drain: [Function: pipeOnDrainFunctionResult],
     error: [Function: onerror],
     close:
      { [Function: bound onceWrapper] listener: [Function: onclose] },
     finish:
      { [Function: bound onceWrapper] listener: [F

CSV file successfully processed


In [5]:
// var nlp = require('compromise') // TODO normalize text strings if needed

In [15]:
// TODO try normalize if needed, skip now

// function preprocess(text) {
//    return nlp(text).normalize({case:false, puctuation:false, plurals:true, parentheses:true, possessives:true, honorifics:true, verbs:true}).out('text');
// }

In [12]:
// var X_preprocessed = X.map(preprocess);

In [13]:
// fs.writeFileSync('./data/X_preprocessed.json', JSON.stringify(X_preprocessed) , 'utf-8'); 

In [14]:
// var X = JSON.parse(fs.readFileSync('./data/X_preprocessed.json'));

### Bag-of-words

In [6]:
// taken and modified from https://github.com/techfort/mimir

In [7]:
function nGram(n) {
  if (typeof n !== 'number' || isNaN(n) || n < 1 || n === Infinity) {
    throw new Error('`' + n + '` is not a valid argument for n-gram')
  }

  return grams

  // Create n-grams from a given value.
  function grams(value) {
    var nGrams = []
    var index

    if (value === null || value === undefined) {
      return nGrams
    }

    value = value.slice ? value : String(value)
    index = value.length - n + 1

    if (index < 1) {
      return nGrams
    }

    while (index--) {
      nGrams[index] = value.slice(index, index + n)
    }

    return nGrams
  }
}

In [8]:
// nGram(2)('hello there how are you')

In [9]:
function tokenize(text) {
    return nGram(2)(text);
}

function extractDictionary(textArray) {
    var dict = {},
      keys = [],
      words;
    textArray = Array.isArray(textArray) ? textArray : [textArray];
    textArray.forEach(function (text) {
      words = tokenize(text);
      words.forEach(function (word) {
        word = word.toLowerCase();
        if (!dict[word] && word !== '') {
          dict[word] = 1;
          keys.push(word);
        } else {
          dict[word] += 1;
        }
      });
    });

    return {
      words: keys,
      dict: dict
    };
  }

function bow(text, vocabulary) {
    var dict = extractDictionary([text]).dict,
      vector = [];

    vocabulary.words.forEach(function (word) {
      vector.push(dict[word] || 0);
    });
    return vector;
}

In [10]:
// stack overflow
function cleanString(input) {
    var output = "";
    for (var i=0; i<input.length; i++) {
        if (input.charCodeAt(i) <= 127) {
            output += input.charAt(i);
        }
    }
    return output;
}

In [11]:
var X = X.map(cleanString);

In [13]:
String.prototype.isEmpty = function() {
    return (this.length === 0 || !this.trim());
};

[Function]

[]

In [12]:
var vocabulary = extractDictionary(X);

In [30]:
function vectorize(text) {
    return bow(text, vocabulary);
}

In [31]:
var X_v = X.map(vectorize);

In [32]:
X_v.length == toxic_y.length

true

In [36]:
var cutout = X_v.length - 1000;
var X_test = X_v.slice(0, cutout);
var y_test = severe_toxic_y.slice(0, cutout);

In [ ]:
fs.writeFileSync('./data/y.json', JSON.stringify(y_test) , 'utf-8'); 

In [ ]:
fs.writeFileSync('./data/X.json', JSON.stringify(X_test) , 'utf-8'); 

### TODO Feature engineering, kitchen sink

In [22]:
// TODO:
// try normalize if needed, skip now
// kitchen sink approach with feature engineering not just bag of words